In [1]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle


# change the working directory to the path where the images are located
p = r"E:\GLO_林口長庚資料集\Process_2022\UNSUPER\iga_mn_data\glomer_features.pkl"

# get the unique labels (from the flower_labels.csv)
df = pd.read_csv('iga_and_mn.csv')



# this list holds all the image filename
glomers = []
data = {}
groups = {}
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
# creates a ScandirIterator aliased as files
path = r"E:\GLO_林口長庚資料集\Process_2022\Data\train\IgA"

with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            glomers.append(file.name)
glomers2 = []
path2 = r"E:\GLO_林口長庚資料集\Process_2022\Data\train\MN"
with os.scandir(path2) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            glomers2.append(file.name)
            


def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

# lop through each image in the dataset
for glomer in glomers:
    # try to extract the features and update the dictionary
    try:
        os.chdir(path)
        feat = extract_features(glomer,model)
        data[glomer] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)

for glomer in glomers2:
    # try to extract the features and update the dictionary
    try:
        os.chdir(path2)
        feat = extract_features(glomer,model)
        data[glomer] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)        

# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)

label = df['label'].tolist()
unique_labels = list(set(label))

# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=100, random_state=22)



1/1 [==============================] - 0s 335ms/step


In [2]:
print(glomers)

['left10010_top11748.jpg', 'left10010_top53538.jpg', 'left10010_top88912.jpg', 'left10018_top109278.jpg', 'left10018_top3058.jpg', 'left10026_top190714.jpg', 'left10046_top33306.jpg', 'left10054_top16532.jpg', 'left10054_top54096.jpg', 'left10072_top128500.jpg', 'left10074_top37416.jpg', 'left10082_top124822.jpg', 'left10094_top113102.jpg', 'left10098_top17756.jpg', 'left10098_top51972.jpg', 'left10114_top13512.jpg', 'left10122_top76150.jpg', 'left10130_top33030.jpg', 'left10150_top10784.jpg', 'left10154_top88184.jpg', 'left10162_top17232.jpg', 'left10162_top29480.jpg', 'left10164_top183312.jpg', 'left10174_top18694.jpg', 'left10184_top18288.jpg', 'left10186_top40548.jpg', 'left10186_top7272.jpg', 'left10190_top63570.jpg', 'left10198_top4598.jpg', 'left10202_top141962.jpg', 'left10204_top3390.jpg', 'left10206_top64962.jpg', 'left10206_top83404.jpg', 'left10210_top53568.jpg', 'left10214_top41198.jpg', 'left10214_top55244.jpg', 'left10214_top82396.jpg', 'left10226_top72502.jpg', 'left102

In [3]:
# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [4]:
def test_data(number_of_cluster, feat, unique_labels, filenames):
    groups = {}

    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=100, random_state=22)
    pca.fit(feat)
    x = pca.transform(feat)
    # cluster feature vectors
    length = len(unique_labels)
    length = number_of_cluster
    kmeans = KMeans(n_clusters=length, random_state=22)
    kmeans.fit(x)

    # holds the cluster id and the images { id: [images] }
    for file, cluster in zip(filenames,kmeans.labels_):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)
    
            
    # this is just incase you want to see which value for k might be the best 
    # sse = []
    # list_k = list(range(3, 50))

    # for k in list_k:
    #     km = KMeans(n_clusters=k, random_state=22)
    #     km.fit(x)
        
    #     sse.append(km.inertia_)

    # # Plot sse against k
    # plt.figure(figsize=(6, 6))
    # plt.plot(list_k, sse)
    # plt.xlabel(r'Number of clusters *k*')
    # plt.ylabel('Sum of squared distance');
    
    return groups

In [5]:
df = pd.read_csv(r'E:\GLO_林口長庚資料集\Process_2022\UNSUPER\GLOMER_UNSUPER\iga_and_mn.csv')
print(df)

                          file  label
0       left10010_top11748.jpg      0
1       left10010_top53538.jpg      0
2       left10010_top88912.jpg      0
3      left10018_top109278.jpg      0
4        left10018_top3058.jpg      0
...                        ...    ...
11394         S1112026 (3).jpg      1
11395         S1112026 (4).jpg      1
11396         S1112026 (5).jpg      1
11397         S1112026 (6).jpg      1
11398         S1112026 (7).jpg      1

[11399 rows x 2 columns]


In [10]:
testing_numbers = [2,3,4,8,10,20,30,50,100]
merged_df = df.copy()
for ii in testing_numbers:
    temp_dict = test_data(ii, feat, unique_labels, filenames)
    list_of_dicts = [{'file': value, f'cluster{ii}': key} for key, values in temp_dict.items() for value in values]

    # Create a DataFrame from the list of dictionaries
    temp_df = pd.DataFrame(list_of_dicts)
    print(temp_df)
    merged_df = merged_df.merge(temp_df, on='file')

c:\Users\VLSI\anaconda3\envs\Tf-ix\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                          file  cluster2
0       left10010_top11748.jpg         0
1       left10010_top53538.jpg         0
2       left10010_top88912.jpg         0
3      left10026_top190714.jpg         0
4       left10054_top16532.jpg         0
...                        ...       ...
11266   left9846_top137186.jpg         1
11267    left9882_top30490.jpg         1
11268    left9914_top74756.jpg         1
11269   left9978_top120590.jpg         1
11270         S0941129 (2).jpg         1

[11271 rows x 2 columns]


c:\Users\VLSI\anaconda3\envs\Tf-ix\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                         file  cluster3
0      left10010_top11748.jpg         0
1      left10010_top53538.jpg         0
2      left10010_top88912.jpg         0
3       left10018_top3058.jpg         0
4      left10054_top16532.jpg         0
...                       ...       ...
11266   left9978_top96766.jpg         1
11267        S0941129 (1).jpg         1
11268        S0941129 (3).jpg         1
11269        S0941129 (5).jpg         1
11270        S0941129 (6).jpg         1

[11271 rows x 2 columns]


c:\Users\VLSI\anaconda3\envs\Tf-ix\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

                          file  cluster30
0       left10010_top11748.jpg         19
1       left10130_top33030.jpg         19
2      left10310_top107698.jpg         19
3       left10324_top40736.jpg         19
4       left10424_top30328.jpg         19
...                        ...        ...
11266    left9330_top28080.jpg          8
11267    left9454_top69428.jpg          8
11268    left9546_top44734.jpg          8
11269    left9598_top30444.jpg          8
11270   left9724_top177668.jpg          8

[11271 rows x 2 columns]


c:\Users\VLSI\anaconda3\envs\Tf-ix\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                         file  cluster50
0      left10010_top11748.jpg          1
1      left10424_top30328.jpg          1
2      left10450_top36562.jpg          1
3      left10530_top38074.jpg          1
4      left10534_top57484.jpg          1
...                       ...        ...
11266  left74670_top59302.jpg         16
11267  left8256_top129830.jpg         16
11268  left84940_top63028.jpg         16
11269   left9410_top83938.jpg         16
11270  left9526_top153808.jpg         16

[11271 rows x 2 columns]


c:\Users\VLSI\anaconda3\envs\Tf-ix\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                          file  cluster100
0       left10010_top11748.jpg          91
1      left10310_top107698.jpg          91
2       left10336_top19884.jpg          91
3       left10530_top38074.jpg          91
4        left10546_top6448.jpg          91
...                        ...         ...
11266         S1001399 (3).jpg          54
11267         S1001399 (4).jpg          54
11268         S0941129 (7).jpg          54
11269         S0941129 (8).jpg          54
11270         S0941129 (9).jpg          54

[11271 rows x 2 columns]


In [11]:
merged_df.to_csv('cluster_result.csv', index=0)

In [12]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

import sys
sys.path.append('E:\casper\OTHER')
from config import get_config 
config = get_config()

def Notification(subject,message):
    content = MIMEMultipart()  
    token = config['token']
    email = config['email']
    content["subject"] = subject
    content["from"] = "VLSI machine"
    content["to"] = config['myemail']
    content.attach(MIMEText(message))
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
        try:
            smtp.ehlo()
            smtp.starttls()
            smtp.login(email, token)
            smtp.send_message(content)
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)
            
mes = """Hi Casper,

    Unsupervised finished. Please check.
"""
sub = "UNSUPER"
Notification(sub,mes)

Complete!


In [40]:
import pandas as pd

# Sample DataFrames
df1 = pd.DataFrame({'value': ['a', 'b', 'c'], 'key': [1, 2, 3]})
df2 = pd.DataFrame({'value': ['c', 'a', 'b'], 'key0': [4, 5, 6]})

# Merge the DataFrames using pd.concat
merged_df = df1.merge(df2, on='value')

print(merged_df)


  value  key  key0
0     a    1     5
1     b    2     6
2     c    3     4


In [49]:
print(len(groups))

50


In [50]:
print(groups)

{1: ['left10010_top11748.jpg', 'left10424_top30328.jpg', 'left10450_top36562.jpg', 'left10530_top38074.jpg', 'left10534_top57484.jpg', 'left10546_top6448.jpg', 'left10758_top90072.jpg', 'left10778_top76070.jpg', 'left10814_top7318.jpg', 'left11010_top51168.jpg', 'left11102_top2876.jpg', 'left11238_top8944.jpg', 'left11276_top4472.jpg', 'left11600_top79252.jpg', 'left11604_top6628.jpg', 'left11616_top80150.jpg', 'left11660_top79682.jpg', 'left11700_top7576.jpg', 'left11720_top95496.jpg', 'left11738_top7050.jpg', 'left11780_top58434.jpg', 'left11792_top36826.jpg', 'left11882_top3574.jpg', 'left12068_top8296.jpg', 'left12192_top10862.jpg', 'left12228_top15578.jpg', 'left12232_top74750.jpg', 'left12272_top2974.jpg', 'left12312_top75538.jpg', 'left12356_top75756.jpg', 'left12628_top62720.jpg', 'left12864_top62512.jpg', 'left12900_top79386.jpg', 'left12910_top79858.jpg', 'left13168_top12852.jpg', 'left13168_top82052.jpg', 'left13180_top38482.jpg', 'left13184_top82590.jpg', 'left13232_top5941

In [51]:
import pandas as pd

data = {1: ['a', 'b', 'c'], 2: ['d', 'e', 'f']}

# Convert the dictionary to a list of dictionaries
list_of_dicts = [{'value': value, 'key': key} for key, values in groups.items() for value in values]

# Create a DataFrame from the list of dictionaries
df1 = pd.DataFrame(list_of_dicts)
df1 = df1.rename(columns={"value":'file', 'key': 'class50'})
print(df1)


                         file  class50
0      left10010_top11748.jpg        1
1      left10424_top30328.jpg        1
2      left10450_top36562.jpg        1
3      left10530_top38074.jpg        1
4      left10534_top57484.jpg        1
...                       ...      ...
22537  left74670_top59302.jpg       16
22538  left8256_top129830.jpg       16
22539  left84940_top63028.jpg       16
22540   left9410_top83938.jpg       16
22541  left9526_top153808.jpg       16

[22542 rows x 2 columns]


In [52]:
# df = pd.read_csv(r'E:\GLO_林口長庚資料集\Process_2022\UNSUPER\GLOMER_UNSUPER\iga_and_mn.csv')
# print(df)

In [53]:
merged_df = merged_df.merge(df1, on='file')

In [54]:
print(merged_df)

                         file  label  class50_x  class50_y
0      left10010_top11748.jpg      0          1          1
1      left10010_top11748.jpg      0          1          0
2      left10010_top53538.jpg      0         49         49
3      left10010_top53538.jpg      0         49          0
4      left10010_top88912.jpg      0         11         11
...                       ...    ...        ...        ...
22793        S1112026 (5).jpg      1         36          0
22794        S1112026 (6).jpg      1         36         36
22795        S1112026 (6).jpg      1         36          0
22796        S1112026 (7).jpg      1         36         36
22797        S1112026 (7).jpg      1         36          0

[22798 rows x 4 columns]
